# Part 1: Power Law Transformation (Gamma Correction)

Simple version in the same style as **Week 04, Lab 01 (1)**.

### Explanation

Gamma correction applies a nonlinear mapping:

$$O = c \cdot (I/255)^{\gamma}$$

- `\gamma < 1` brightens dark regions
- `\gamma > 1` darkens the image

# Task 1: Gamma Correction on Images

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os

## 1. Define the Gamma Correction Function

In [ ]:
def gamma_correction(I_in, c, gamma):
    I_norm = I_in.astype('float32') / 255.0
    I_gamma = c * (I_norm ** gamma)
    I_out = np.clip(I_gamma * 255.0, 0, 255).astype('uint8')
    return I_out

# 2. Load Images and Apply Gamma Correction

In [ ]:
folder = 'lab04_assets/images'
out_root = 'lab04_outputs'
out_gamma = os.path.join(out_root, 'task1_gamma')
out_contrast = os.path.join(out_root, 'task2_contrast')
os.makedirs(out_root, exist_ok=True)
os.makedirs(out_gamma, exist_ok=True)
os.makedirs(out_contrast, exist_ok=True)

image_files = sorted([f for f in os.listdir(folder) if f.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp'))])
if not image_files:
    raise FileNotFoundError(f'No images found in {folder}')

gamma_values = {'1.png': 0.5, '2.png': 0.8, '3.png': 0.6, '4.png': 0.6, '5.jpg': 0.2}
why = {
    '1.png': 'dark image, moderate brightening',
    '2.png': 'already okay, slight brightening',
    '3.png': 'shadow details needed boost',
    '4.png': 'dim image, gamma 0.6 looked balanced',
    '5.jpg': 'very underexposed, strong brightening',
}

print('Images:', image_files)

for fname in image_files:
    img = cv2.imread(os.path.join(folder, fname), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    g = gamma_values.get(fname, 0.8)
    out = gamma_correction(img, c=1.0, gamma=g)

    cv2.imwrite(os.path.join(out_gamma, f"gamma_{str(g).replace('.', 'p')}_{fname}"), out)

    plt.figure(figsize=(10, 3))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title(f'Original: {fname}')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(out, cmap='gray', vmin=0, vmax=255)
    plt.title(f'Gamma={g}')
    plt.axis('off')
    plt.tight_layout()
    plt.show()

    print(f"{fname}: gamma={g} -> {why.get(fname, 'chosen visually')}")

## 3. Normalized Save Issue (cv2.imwrite)

In [ ]:
sample = cv2.imread(os.path.join(folder, image_files[0]), cv2.IMREAD_GRAYSCALE)
sample_norm = sample.astype('float32') / 255.0
wrong = sample_norm.astype('uint8')
right = np.clip(sample_norm * 255.0, 0, 255).astype('uint8')

cv2.imwrite(os.path.join(out_root, 'task1_wrong_save_cast_uint8.png'), wrong)
cv2.imwrite(os.path.join(out_root, 'task1_correct_save_scaled_uint8.png'), right)

plt.figure(figsize=(12, 3))
plt.subplot(1, 3, 1)
plt.imshow(sample, cmap='gray', vmin=0, vmax=255)
plt.title('Original')
plt.axis('off')

plt.subplot(1, 3, 2)
plt.imshow(wrong, cmap='gray', vmin=0, vmax=255)
plt.title('Wrong cast')
plt.axis('off')

plt.subplot(1, 3, 3)
plt.imshow(right, cmap='gray', vmin=0, vmax=255)
plt.title('Correct scale')
plt.axis('off')

plt.tight_layout()
plt.show()

# Part 2: Contrast Stretching

### Formula

$$i_{out} = \left(\frac{s_{max}-s_{min}}{r_{max}-r_{min}}\right)(i_{in}-r_{min}) + s_{min}$$

# 1. Contrast Stretching Function

In [ ]:
def contrast_stretching(i_in, s_min=0, s_max=255):
    i_in = i_in.astype('float32')
    r_min = np.min(i_in)
    r_max = np.max(i_in)

    if r_max == r_min:
        return np.zeros_like(i_in, dtype='float32')

    i_out = ((s_max - s_min) / (r_max - r_min)) * (i_in - r_min) + s_min
    return i_out

# 2. Non-normalized Images
Using `(s_min=255, s_max=0)` and `(s_min=0, s_max=255)`

In [ ]:
for fname in image_files:
    img = cv2.imread(os.path.join(folder, fname), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    given = contrast_stretching(img, s_min=255, s_max=0)
    standard = contrast_stretching(img, s_min=0, s_max=255)

    given_u8 = np.clip(given, 0, 255).astype('uint8')
    standard_u8 = np.clip(standard, 0, 255).astype('uint8')

    cv2.imwrite(os.path.join(out_contrast, f'given_0_255_{fname}'), given_u8)
    cv2.imwrite(os.path.join(out_contrast, f'standard_255_0_{fname}'), standard_u8)

    plt.figure(figsize=(12, 3))
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 3, 2)
    plt.imshow(given_u8, cmap='gray', vmin=0, vmax=255)
    plt.title('Given (0,255)')
    plt.axis('off')

    plt.subplot(1, 3, 3)
    plt.imshow(standard_u8, cmap='gray', vmin=0, vmax=255)
    plt.title('Standard (255,0)')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# 3. Normalized Images (0-1)
Using `(s_min=1, s_max=0)` then scaling back to 0-255 for saving

In [ ]:
for fname in image_files:
    img = cv2.imread(os.path.join(folder, fname), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    img_norm = img.astype('float32') / 255.0
    out_norm = contrast_stretching(img_norm, s_min=1, s_max=0)
    out_norm_u8 = np.clip(out_norm * 255.0, 0, 255).astype('uint8')

    cv2.imwrite(os.path.join(out_contrast, f'norm_given_0_1_{fname}'), out_norm_u8)

    plt.figure(figsize=(8, 3))
    plt.subplot(1, 2, 1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 2, 2)
    plt.imshow(out_norm_u8, cmap='gray', vmin=0, vmax=255)
    plt.title('Normalized (0,1)')
    plt.axis('off')

    plt.tight_layout()
    plt.show()

# 4. Other Combinations
Try `(s_min=0, s_max=50)` and `(s_min=100, s_max=160)`

In [ ]:
combinations = [(0, 50), (100, 160)]

for fname in image_files:
    img = cv2.imread(os.path.join(folder, fname), cv2.IMREAD_GRAYSCALE)
    if img is None:
        continue

    plt.figure(figsize=(12, 3))
    plt.subplot(1, 3, 1)
    plt.imshow(img, cmap='gray', vmin=0, vmax=255)
    plt.title('Original')
    plt.axis('off')

    k = 2
    for s_min, s_max in combinations:
        out = contrast_stretching(img, s_min=s_min, s_max=s_max)
        out_u8 = np.clip(out, 0, 255).astype('uint8')
        cv2.imwrite(os.path.join(out_contrast, f'case_{s_min}_{s_max}_{fname}'), out_u8)

        plt.subplot(1, 3, k)
        plt.imshow(out_u8, cmap='gray', vmin=0, vmax=255)
        plt.title(f'({s_min},{s_max})')
        plt.axis('off')
        k += 1

    plt.tight_layout()
    plt.show()

print('Done. Outputs saved in lab04_outputs/')